# Testing Derivatives

In [11]:
import chemistry as chem
import autohf as hf
import numpy as np
import autograd
from autograd.differential_operators import make_jvp
import autograd.numpy as anp
from pennylane import qchem
import pennylane as qml
import bigvqe as bv

np.set_printoptions(linewidth=800, threshold=200)

In [12]:
molecule = chem.BeH2()
R = molecule.optimal_coordinates # Hartree-Fock geometry
structure = molecule.structure # Molecular symbols

num_elecs = 6
num_orbitals = 7
charges = [4, 1, 1]
wires = list(range(12))

core, active = qchem.active_space(6, 7, active_electrons=4, active_orbitals=6) # Prepares active space

In [13]:
basis_set = []
A1, A2, A3 = hf.basis_set_params("sto-3g", structure)
for func in A1 + A2 + A3:
    L, exp, coeff = func
    basis_set.append(hf.AtomicBasisFunction(L, C=anp.array(coeff), A=anp.array(exp)))

hf_b = [basis_set[0:5], [basis_set[5]], [basis_set[6]]]

In [10]:
H = chem.hamiltonian(molecule.structure, hf_b, 6, charges, wires, core=core, active=active)

# Defines Hamiltonian and sparse Hamiltonian at HF geometry
H_r = H(R)
H_r_sparse = qml.SparseHamiltonian(qml.utils.sparse_hamiltonian(H_r), wires=wires)

In [14]:
# Defines the gradient of the Hamiltonian
convert = lambda x : anp.array([0.0, 0.0, 0.0, 0.0, x[0], 0.0, 0.0, -x[0], 0.0])
H_sparse = lambda x : qml.SparseHamiltonian(chem.hamiltonian_sparse(hf_b, 6, charges, core=core, active=active)(convert(x)), wires=wires)

def dH(x):
    vector = np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0])
    dH = chem.d_hamiltonian_sparse(hf_b, 6, charges, core=core, active=active)
    return [dH(convert(x), vector)]

dH_sparse = lambda x : [qml.SparseHamiltonian(h, wires=wires) for h in dH(x)]